Overall plan:
- Get sampled values from image
- Center image on source peak
- 5 arcsec x 5 arcsec
- Interpolate vertical and horizontal directions to "smooth" image
- Shove into SQL database
- Call this function in make_catalog

In [ ]:
from astropy.coordinates import Angle
from astropy.io import fits
from find_source import fits_data_index

In [ ]:
fits_file = '' # change this to actual path for testing


##### BELOW IS COPIED FROM region_stats #####
i = fits_data_index(fits_file)

# open FITS file
try:
    file = fits.open(fits_file)
except:
    print(f'Unable to open {fits_file}')

# extract data array
info = file[i]
data = info.data

In [ ]:
data_array = data[0]

In [ ]:
header_data = fits.getheader(fits_file)

pixel_scale = Angle(header_data['CDELT1'], header_data['CUNIT1']).to_value('arcsec')
x_dim = header_data['NAXIS1']
y_dim = header_data['NAXIS2']

center = (round(x_dim/2), round(y_dim/2))

In [ ]:
# x_coords.append((ext_peak_coords[i][0] - center[0]) * pixel_scale)


In [ ]:
sampled_values = {} # dictionary of sampled data points

Should test to decide how "finely" I want to interpolate things

arguments:
- peak coords (normalized)
- pixel scale
- x-axis length
- y-axis length
- fits file path

In [ ]:
def interpolated_value(x):
    # TODO: assert k between -1 and N + 1
    # TODO: find x_k and x_k+1 s.t. x_k <= x <= x_k+1
    # TODO: grab c values from dictionary
    # TODO: plug everything into interpolation function
    # TODO: return value
    pass